In [ ]:
from typing import Iterator
from math import log, sin, pi
from queue import Queue

def lcg(seed: int, m: int=2**32, a: int=1664525, c: int=1013904223) -> Iterator[int]:
    """
    This generator implements the Linear Congruential Generator algorithm
    :param m: the modulus, a positive integer constant
    :param a: the multiplier, a non-negative integer constant < m
    :param c: the increment, a non-negative integer constant < m
    :param seed: the starting state of the LCG. It is used to initialize the pseudo-random number sequence
    :return: a non-negative integer in [0, m-1] representing the i-th state of the generator
    """
    x = seed
    while True:
        x = (a * x + c) % m
        yield x/m

def xorshift_64(seed: int) -> Iterator[int]:
    """
    This generator implements the XORShift algorithm.
    :param seed: the initial state of the generator. Should be a non-zero integer.
    :return: pseudo-random integers generated by the XORShift algorithm.
    """
    x = seed if seed != 0 else 1
    while True:
        x ^= (x << 13) & 0xFFFFFFFFFFFFFFFF
        x ^= (x >> 7) & 0xFFFFFFFFFFFFFFFF
        x ^= (x << 17) & 0xFFFFFFFFFFFFFFFF
        x &= 0xFFFFFFFFFFFFFFFF
        yield x/0xFFFFFFFFFFFFFFFF


def rotl(x: int, k: int) -> int:
    return ((x << k) | (x >> (64 - k))) & 0xFFFFFFFFFFFFFFFF

def xoshiro_64(seed: list[int]) -> Iterator[int]:
    """
    xoshiro256++ generator
    :param seed: list of 4 uint64 integers, the internal state (cant be all 0)
    :return: pseudo-random 64-bit integers
    """
    s = seed.copy()
    if len(s) != 4 or all(x == 0 for x in s):
        raise ValueError("Seed must be a list of 4 non-zero uint64 integers")

    while True:
        result = (rotl(s[0] + s[3], 23) + s[0]) & 0xFFFFFFFFFFFFFFFF
        t = (s[1] << 17) & 0xFFFFFFFFFFFFFFFF

        s[2] ^= s[0]
        s[3] ^= s[1]
        s[1] ^= s[2]
        s[0] ^= s[3]

        s[2] ^= t
        s[3] = rotl(s[3], 45)

        yield result/0xFFFFFFFFFFFFFFFF
        
def Poisson_adelgazamiento_mejorado(T, generator):
    interv = [4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48]
    lamda = [12.5, 27.5, 35, 35, 27.5, 12.5, 12.5, 27.5, 35, 35, 27.5, 12.5]
    j = 0 #recorre subintervalos.
    t =-log (1 - next(generator) ) / lamda[j]
    NT = 0
    Eventos = []
    while t <= T:
        if t <= interv[j]:
            V = next(generator)
            if V < 20 + 15*sin((t-6)*2*pi/24) / lamda[j]:
                Eventos.append(t)
                t +=-log(1- next(generator)) / lamda[j]
        else: #t > interv[j]
            t = interv[j] + (t- interv[j]) * lamda[j] / lamda[j + 1]
            j += 1
    return Eventos

class Cliente:
    def __init__(self, tiempo_llegada: float):
        self.tiempo_llegada = tiempo_llegada
        self.tiempo_espera = 0.0
        self.tiempo_en_sistema = 0.0
    

def simulacion(generator):
    T = 48
    Eventos = Poisson_adelgazamiento_mejorado(T, generator)
    j = 0
    tiempo_actual = 0
    cola = Queue()
    datos_clientes = []
    datos_cola = {}
    while True:
        if cola.empty():
            if j < len(Eventos):
                tiempo_actual = Eventos[j]
                cliente = Cliente(tiempo_actual)
                cola.put(cliente)
                j += 1
                datos_cola[tiempo_actual] = cola.qsize()
            else:
                break
        else:
            cliente_atendido = cola.get()
            cliente_atendido.tiempo_espera = tiempo_actual - cliente_atendido.tiempo_llegada
            
            tiempo_servicio = -log(1 - next(generator)) / 35
            cliente_atendido.tiempo_en_sistema = cliente_atendido.tiempo_espera + tiempo_servicio
            tiempo_actual += tiempo_servicio
            
            datos_clientes.append(cliente_atendido)
            datos_cola[tiempo_actual] = cola.qsize()
            
            while j < len(Eventos) and Eventos[j] < tiempo_actual:
                cliente = Cliente(Eventos[j])
                cola.put(cliente)
                datos_cola[Eventos[j]] = cola.qsize()
                j += 1
            
    return datos_clientes, datos_cola


        
        
        
lcg_generator = lcg(seed=12)
datos_clientes, datos_cola = simulacion(lcg_generator)

print("Datos de los clientes:")
print(f"Tiempo promedio de espera: {sum(cliente.tiempo_espera for cliente in datos_clientes) / len(datos_clientes):.2f} horas")
print(f"Maximo tiempo de espera: {max(cliente.tiempo_espera for cliente in datos_clientes):.2f} horas")

print("Datos de la cola:")
print(f"Tamaño promedio de la cola: {sum(datos_cola.values()) / len(datos_cola):.2f}")
print(f"Tiempo de maximo tamaño de la cola: {max(datos_cola, key=datos_cola.get):.2f} horas")
print(f"Maximo tamaño de la cola: {max(datos_cola.values()):.2f}")